# Checklist - Análise de Sentimentos IMDB
### SckitLearn Model Logistic Regression

Author: Felipe Thamay

In [1]:
import model_linear_regression
import checklist

from sklearn.pipeline import make_pipeline
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.test_suite import TestSuite
from checklist.expect import Expect
from checklist.pred_wrapper import PredictorWrapper

c:\users\felipe\anaconda3\envs\checklist3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### MFT

O modelo deve ser capaz de distinguir questões relacionadas a emoções positivas e negativas.

In [2]:
suite = TestSuite()
editor = Editor(language='english')

In [3]:
film_noun = ['story', 'tale', 'description', 'picture', 'sketch', 'painting', 'narrative', 'panel', 'narrative', 'report',
'report', 'fable', 'outline', 'portrait', 'trace', 'review', 'grove', 'relation', 'tradition', 'novel', 'story',
'exhibition', 'saga', 'raconto', 'drawing', 'outline', 'record', 'fiction', 'biography', 'legend', 'guide', 'script', 'defeat',
'route', 'script', 'itinerary', 'novelist', 'novelist', 'historian', 'narrator', 'chronist', 'humorist', 'story writer', 'anesdotista',
'Biographer', 'Adventure', 'Biographic', 'Comedy', 'Dramatic Comedy', 'Romantic Comedy', 'Historical', 'Drama', 'Action',
'Science fiction', 'Horror']
editor.add_lexicon('film_noun', film_noun, overwrite=True)

In [4]:
pos_adj = ['good','wonderful','pleasure','pleasant','surprising','funny','great','excellent','amazing',
             'extraordinary','beautiful','fantastic','exceptional','perfect','fun','happy','lovely',
             'brilliant','exciting', 'sweet','wonderful','ok']
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)

In [5]:
neg_adj = ['boring','tiring','monotonous','bad','terrible',"didn't like it",'awful','cliché','tiring',
             'strange', 'rough', 'terrible','unfortunate','average','difficult','poor','sad','frustrating',
             'difficult','lame','nasty','annoying','frightening','terrible','ridiculous','terrible','ugly',
             'unpleasant']
editor.add_lexicon('neg_adj', neg_adj, overwrite=True)

In [6]:
neutral_adj = ['American','international','commercial','British','private','Italian','Indian','Australian',
                'Israeli','Brazilian']
editor.add_lexicon('neutral_adj', neutral_adj, overwrite=True)

In [7]:
article = ['o','a','os','as']
editor.add_lexicon('article', article, overwrite=True)

In [8]:
pos_verb_present = ['like','enjoy','appreciate','love','recommend','admire','value','welcome']
neg_verb_present = ['hate','dislike','repent','hate','dread','dislike']
neutral_verb_present = ['see','find']
pos_verb_past = ['liked','appreciated','loved','admired','valued']
neg_verb_past = ['hated',"didn't like it",'sorry','feared','disdained']
neutral_verb_past = ['saw','found']
demo_pron = ['This','This','This','This','That','That']
pred = ['is', 'was', 'like']
pers_pron = ['I','You','He','We','You','They']
                 
editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)
editor.add_lexicon('demo_pron', demo_pron, overwrite=True)
editor.add_lexicon('pred', pred, overwrite=True)
editor.add_lexicon('pers_pron', pers_pron, overwrite=True)

In [9]:
t = editor.template(
    '{demo_pron} {film_noun} {pred} {pos_adj}.', 
    remove_duplicates=True, 
    nsamples=300, 
    labels=1)

test = MFT(
    **t, name='Palavras carregadas de sentimento no contexto', 
    capability = 'NER',
    description='Retorne positivo ou negativo, mesmo que tenha erro de digitação')

suite.add(test, overwrite=True)

In [10]:
pipe = make_pipeline(model_linear_regression.vect, model_linear_regression.model_lr)

In [11]:
# PredictorWrapper
predictor = PredictorWrapper().wrap_softmax(pipe.predict_proba)

In [12]:
suite.run(predictor, overwrite=True)
suite.visual_summary_table()

Running Palavras carregadas de sentimento no contexto
Predicting 300 examples
Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Palavras carregadas …

### INV (INVariance test - Teste de invariância)

Se você tiver duas frases com a mesma entidade nomeada, alterar a entidade em ambas não deve alterar se as perguntas são duplicadas ou não.
Vamos escrever um INV para isso.

In [13]:
neutral_words = set(
    ['.','o','O',',','a','A','and','from','to','this','that','in',' this','to',
      'you','there','or','a','by','on','film','mine','in','from','has','with','was ',
      'in','this','get','from','this','American','international','commercial',
      'British','private','Italian','Indian','Australian','Israeli','Brazilian',
      'see','find','saw','found'])
forbidden = set(['No','no','No','no','Nothing','nothing','without','but'] + pos_adj + neg_adj + pos_verb_present 
                + pos_verb_past + neg_verb_present + neg_verb_past)
def change_neutral(d):
    examples = []
    subs = []
    words_in = [x for x in d.capitalize().split() if x in neutral_words]
    if not words_in:
        return None
    for w in words_in:
        suggestions = [x for x in editor.suggest_replace(d, w, beam_size=5, words_and_sentences=True) if x[0] not in forbidden]
        examples.extend([x[1] for x in suggestions])
        subs.extend(['%s -> %s' % (w, x[0]) for x in suggestions])
    if examples:
        idxs = np.random.choice(len(examples), min(len(examples), 10), replace=False)
        return [examples[i] for i in idxs]

In [14]:
t = Perturb.perturb(
    model_linear_regression.df_test.review.array, 
    Perturb.add_typos, 
    nsamples=200, 
    typos=3)

test = INV(
    **t, 
    name='Mude palavras neutras com BERT', 
    capability='NER', 
    description='Altere um conjunto de palavras neutras por outras palavras neutras apropriadas ao contexto')

suite.add(test, overwrite=True)

In [15]:
suite.run(predictor, overwrite=True)
suite.visual_summary_table()

Running Palavras carregadas de sentimento no contexto
Predicting 300 examples
Running Mude palavras neutras com BERT
Predicting 400 examples
Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Palavras carregadas …

### DIR (DIRectional expectation test - Teste de expectativa direcional)

In [16]:
df0 = model_linear_regression.df_test[model_linear_regression.df_test['sentiment'] == 'neg']
df1= model_linear_regression.df_test[model_linear_regression.df_test['sentiment'] == 'pos']

In [17]:
expect_fn = Expect.monotonic(label=1, increasing=True, tolerance=0.05)

In [18]:
t = Perturb.perturb(
    df1.review.array, 
    Perturb.add_typos, 
    nsamples=200)
name = 'Mudar o nome em uma das questões'
desc = 'Pegue os pares que foram originalmente previstos como duplicados, altere o nome em um deles e espere que a nova previsão não seja duplicada'

test = DIR(
    **t, 
    expect=expect_fn, 
    name=name, 
    description=desc, 
    capability='NER')

suite.add(test, overwrite=True)

In [19]:
suite.run(predictor, overwrite=True)
suite.visual_summary_table()

Running Palavras carregadas de sentimento no contexto
Predicting 300 examples
Running Mude palavras neutras com BERT
Predicting 400 examples
Running Mudar o nome em uma das questões
Predicting 400 examples
Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Palavras carregadas …